In [33]:
import sys
import pandas as pd
sys.path.insert(0, './Moduulit')
pd.options.mode.chained_assignment = None

from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets


# Importtaus tässä muodossa
from database_connection import database_query
from outliers import find_outliers, draw_histogram
from clean_dataframe import clean_dataframe
from draw_node_amount import *
from write_df_to_database import write_df_to_mariadb
from xy_diskretisointi import xy_to_grid
from reittien_erottelu import *
from isolation_forest import *
from sisään_uloskäynnit import *
from horrostila import *
from widgets import *
from plot_lapimenoajat import *
from histogrammi_3D import *
# tai from ... import *

In [34]:
df = database_query("SELECT * FROM hyvatReititDB limit 1000000")
df.head()

,ajokerta,node_id,timestamp,x,y,grid_id,kesto,dayofweek,current_hour
0,1,1,2020-06-24 15:20:12,22,11,352,64858.412,2,8
1,1,1,2020-06-24 15:20:13,22,11,352,64858.412,2,8
2,1,1,2020-06-24 15:20:14,22,11,352,64858.412,2,8
3,1,1,2020-06-24 15:20:21,26,12,386,64858.412,2,8
4,1,1,2020-06-24 15:20:22,25,12,385,64858.412,2,8


In [35]:
import numpy as np
import matplotlib.pyplot as plt


In [36]:
X= df[['x', 'y']]

In [37]:
X

,x,y
0,22,11
1,22,11
2,22,11
3,26,12
4,25,12
...,...,...
999995,25,16
999996,25,16
999997,25,17
999998,25,16


In [43]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5).fit(X)
centroids = kmeans.cluster_centers_
print(centroids)

plt.figure(figsize=(20,10))
plt.plot(df['x'], df['y'], c= kmeans.labels_.astype(float))
plt.scatter(centroids[:, 0], centroids[:, 1], c='red', s=50)
plt.show()

[[22.63317913 15.66119209]
 [12.37299048 15.40232776]
 [21.34847874 23.93322588]
 [19.74814548  9.43505475]
 [ 7.67768819 23.35164749]]


ValueError: RGBA sequence should have length 3 or 4

<Figure size 1440x720 with 1 Axes>